In [2]:
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.layers.core import Lambda
from keras.layers import Input
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
from noise_functions_dl import GradientDescentDL, gradientDescentFunc
import os
import numpy as np

Using TensorFlow backend.


In [3]:
input_tensor = Input(shape=(224, 224, 3))
tf_inputs = Lambda(lambda x: preprocess_input(x, mode='tf'))(input_tensor)
caffe_inputs = Lambda(lambda x: preprocess_input(x, mode='caffe'))(input_tensor)

base_inception = InceptionV3(input_tensor=input_tensor, weights="imagenet", include_top=True)
inception = Model(inputs=input_tensor, outputs=base_inception(tf_inputs))

base_resnet = ResNet50(input_tensor=input_tensor, weights="imagenet", include_top=True)
resnet = Model(inputs=input_tensor, outputs=base_resnet(caffe_inputs))

base_inceptionresnet = InceptionResNetV2(input_tensor=input_tensor, weights="imagenet", include_top=True)
inceptionresnet = Model(inputs=input_tensor, outputs=base_inceptionresnet(tf_inputs))

base_vgg = VGG19(input_tensor=input_tensor, weights="imagenet", include_top=True)
vgg = Model(inputs=input_tensor, outputs=base_vgg(caffe_inputs))

# [vgg, inceptionresnet, resnet, inception]
models = [vgg, inceptionresnet, resnet, inception]

for model in models:
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
data_path = "imagenet_data"
X_exp = np.load(data_path + "/" + "X_exp.npy")[:50]
Y_exp = np.load(data_path + "/" + "Y_exp.npy")[:50]
Target_exp = np.load(data_path + "/" + "Target_exp.npy")[:50]

In [34]:
baseline_folder = "imagenet_baselines_2"
noise_baselines = [np.load(baseline_folder + "/imagenet_baseline_untargeted_{}/noise.npy".format(i)) 
                   for i in xrange(5)]

In [11]:
[model.evaluate(X_exp, Y_exp) for model in models]

100/100 [==============================] - 27s 266ms/step


[[0.09790555447340012, 1.0],
 [0.0023349857737775892, 1.0],
 [0.09216954857110977, 1.0],
 [0.02559835523366928, 1.0]]

In [35]:
baseline_results = [[model.evaluate(X_exp + noise, Y_exp)[1] for model in models] for noise in noise_baselines]

50/50 [==============================] - 18s 351ms/step


In [38]:
for res in baseline_results:
    print res

[0.12000000014901162, 1.0, 1.0, 0.98]
[1.0, 0.3000000047683716, 1.0, 1.0]
[1.0, 1.0, 0.0, 0.98]
[1.0, 1.0, 1.0, 0.0]
[0.24000000029802324, 0.8799999928474427, 0.10000000029802322, 0.5799999976158142]


In [23]:
min([max(res) for res in baseline_results])

0.8799999928474427

In [25]:
for res in baseline_results:
    print res

[1.0, 1.0, 0.0, 0.98]
[1.0, 1.0, 1.0, 0.0]
[0.24000000029802324, 0.8799999928474427, 0.10000000029802322, 0.5799999976158142]
[1.0, 0.3000000047683716, 1.0, 1.0]
[0.12000000014901162, 1.0, 1.0, 0.98]


In [18]:
mwu_noise = np.load("deepLearning-imagenet-untargeted-300.0-2-4-2047/noise.npy")[0]

In [24]:
mwu_results = [model.evaluate(X_exp + mwu_noise, Y_exp)[1] for model in models]

50/50 [==============================] - 13s 252ms/step


In [26]:
mwu_results

[0.1600000001490116, 0.6600000071525574, 0.0, 0.18000000178813935]

In [27]:
max(mwu_results)

0.6600000071525574

# Targeted Noise

In [6]:
mwu_targeted_noise = mwu_noise = np.load("deepLearning-imagenet-targeted-3000.0-2-7-316/noise.npy")[0]

In [7]:
mwu_targeted_noise.shape

(50, 224, 224, 3)

missing ensemble baseline for targeted noise, ensemble is about to finish so I'll run these then

In [10]:
baseline_folder = "imagenet_baselines_targeted"
targeted_noise_baselines = [np.load(baseline_folder + "/imagenet_baseline_2_targeted_{}/noise.npy".format(i)) 
                            for i in xrange(5)]

In [11]:
baseline_results_t = [[model.evaluate(X_exp + mwu_targeted_noise, Target_exp)[1] for model in models] 
                      for noise in targeted_noise_baselines]

50/50 [==============================] - 11s 213ms/step


In [12]:
baseline_results_t

[[1.0, 0.0, 0.0, 0.0],
 [0.0, 0.020000000149011612, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.7200000071525574],
 [0.16000000178813933, 0.0, 0.3000000023841858, 0.1800000002980232]]

In [13]:
mwu_results_t = [model.evaluate(X_exp + mwu_targeted_noise, Target_exp)[1] for model in models]

50/50 [==============================] - 11s 216ms/step


In [15]:
mwu_results_t

[0.5200000011920929,
 0.06000000014901161,
 0.6800000071525574,
 0.5200000023841858]